<a href="https://colab.research.google.com/github/teena0401/Webscrapping-Indeed-Website/blob/main/Indeed_(Beautiful_Soup).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import sqlite3

In [ ]:
def make_indeed_url(search_job,search_location):
  job = search_job.replace('','%+')
  location = search_location.replace('','%+')
  indeed_job_url = f'https://malaysia.indeed.com/jobs?q={job}&l={location}'
  return indeed_job_url

  https://www.jobstreet.com.my/en/job-search/data-engineer-jobs/
  

In [ ]:
def scrape_job_card(job_meta):
  try: job_title = job_meta.find('h2',{'class':'jobTitle'}).get_text().lstrip('new\n')
  except: job_title = "No Job Title Found"
  try: company_name = job_meta.find('span',{"class":"companyName"}).get_text()
  except: company_name = "No Company Name"
  try:company_location = job_meta.find('div',{'class':"companyLocation"}).get_text()
  except: company_location = "No Location"
  try:estimated_salary = job_meta.find('div',{'class':'metadata salary-snippet-container'}).get_text()
  except: estimated_salary = "No estimated Salary"
  return job_title, company_name, company_location, estimated_salary

In [ ]:
def scrape_job_description(job_desc_href):
  try: 
    page = web_scrape_api_call(job_desc_href)
    soup = BeautifulSoup(page.content,'html.parser')
    job_desc = soup.find(id='jobDescriptionText')
    job_desc=job_desc.text.replace('\n','').replace('\r','')
  except: job_desc = 'No Job Description'
  return job_desc

In [ ]:
def scrape_job_page_meta(job_page_html):
    page_soup = BeautifulSoup(job_page_html.text,'lxml')
    df_columns = ['job_title','company_name','company_location','est_salary','job_href','job_desc']
    obs_df = (pd.DataFrame(columns = df_columns))
    for job in page_soup.find_all('div',{"id":"mosaic-provider-jabcards"}):
      for href_post in job.find_all('a',href=True):
        if href_post in job.find_all('a',href=True):
           job_desc_href = 'https://www.indeed.com'+ str(href_post['href'])
           job_desc = scrape_job_description(job_desc_href)
           for job_meta in href_post.find_all('div',{"class":"job_seen_beacon"}):
              job_title,company_name,company_location,estimated_salary =scrape_job_card(job_meta)
              print(f'{job_title},{job_desc_href}')
              job_dict ={'job_title':job_title,
                          'company_name':[company_name],
                            'est_salary': [estimated_salary],
                             'job_href':[job_desc_href],
                                 'job_desc':[job_desc]}
              i_df = pd.DataFrame.from_dict(job_dict)
              jobs_df = jobs_df.append(j_df,ignore_index=True)
              return jobs_df


In [ ]:
def job_loc_scrape_loop(job_list,loc_list,job_age):
  date= dt.datetime.today().strftime('%Y-%m-%d')
  for job in job_list:
    for loc in loc_list:
      print(f'Scraping:{job}{loc}')
      indeed_url = make_indeed_url(job,loc,job_age)
      indeed_response = web_scrape_api_call(indeed_url)
      result_df = scrape_job_page_meta(indeed_response)
      result_df['retrieve_date']=date
      print('dumping to sql')
      sql_dump(result_df,'data/jobs','indeed_jobs')
      return none 

In [ ]:
def sql_dump(df,db,table):
  con = sqlite3.connect(db)
  df.to_sql(table,con,if_exists='append')
  con.close()
  

In [ ]:
def web_scrape_api_call(url_to_scrape):
  url = "https://api.webscrapingapi.com/v1"
  params ={"api_key":WebScrapingAPIkey,
           "url":url_to_scrape}
  response = requests.request('GET',url,params =params)
  return response